<a href="https://colab.research.google.com/github/ManciSee/Progetto_TMDB/blob/main/Progetto_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis IMDB Film Reviews

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from urllib.request import urlopen as uRequest

In [ ]:
from bs4 import BeautifulSoup as soup

url = uRequest("https://www.imdb.com/search/title/?groups=top_100")
page_html = url.read()
page_soup = soup(page_html)

containers = page_soup.findAll('div', {'class' : 'lister-item mode-advanced'})
container = containers[0]
print(container)


# IMDB stampa i titoli in cinese

In [ ]:
!pip install translate

# Pulizia totale
Pulizia titolo e nome che avevano caratteri speciali come `\n1.\nOppenheimer\n(I)` e `2023)\n` e order come `1.`



In [ ]:
# Funzione per pulire il titolo e l'anno
def clean_title(raw_title):
    # Rimuovi il numero iniziale seguito da un punto e uno spazio
    clean_title = raw_title.split('.', 1)[-1].strip()
    # Rimuovi l'anno tra parentesi alla fine
    clean_title = clean_title.split('(')[0].strip()
    return clean_title

def clean_year(raw_year):
    # Rimuovi eventuali spazi bianchi iniziali e finali
    clean_year = raw_year.strip()
    # Rimuovi il carattere "\n" alla fine
    clean_year = clean_year.rstrip('\n')
    # Rimuovi l'ultima parentesi
    if clean_year.endswith(')'):
        clean_year = clean_year[:-1]
    return clean_year

def clean_order(raw_order):
    # Rimuovi il punto alla fine dell'elemento "order"
    clean_order = raw_order.rstrip('.')
    return clean_order

def clean_review_title(raw_title):
    clean_title = raw_title.strip('\n')
    return clean_title


# Tiriamoci tutte cose

In [ ]:
from bs4 import BeautifulSoup as soup
from translate import Translator
import requests

# Funzione per tradurre il testo dal cinese all'inglese
def translate_to_english(text):
    translator = Translator(to_lang="zh", from_lang="en")
    translation = translator.translate(text)
    return translation

def extract_link_from_h3(h3_element):
    # Cerca l'elemento <a> all'interno dell'elemento h3
    a_element = h3_element.find('a')
    if a_element:
        link = a_element.get('href')
        return link
    return None

def scrape_review_page(review_url):
    page_html = requests.get(review_url).text
    page_soup = soup(page_html, 'html.parser')
    return page_soup

def scrape_page(url):
    page_html = requests.get(url).text
    page_soup = soup(page_html, 'html.parser')

    containers = page_soup.findAll('div', {'class': 'lister-item mode-advanced'})

    _records = []

    main_page = 1

    for container in containers:
        c_title = container.findAll('h3', {'class': 'lister-item-header'})[0].text
        c_order = container.findAll('span', {'class': 'lister-item-index unbold text-primary'})[0].text
        c_year = c_title[-6:]
        c_title = c_title.replace(c_order + ' ', '')[:-7]
        c_link = extract_link_from_h3(container.find('h3', {'class': 'lister-item-header'}))

        c_review_link = 'https://www.imdb.com' + c_link + 'reviews' if c_link else None  # Add the IMDb domain to the review link

        if c_link and c_link.startswith('/title/'):
            c_link = 'https://www.imdb.com' + c_link

        c_title = clean_title(c_title)
        c_year = clean_year(c_year)
        c_order = clean_order(c_order)

        # Translate the title from Chinese to English if needed
        # c_title_english = translate_to_english(c_title)

        # Access the review page and parse it
        if c_review_link:
            review_page_soup = scrape_review_page(c_review_link)

            page = 1
            while True:
                try:
                    load_more = review_page_soup.find_all('button', {'class': 'ipl-load-more__button'})
                    if load_more and len(load_more) > 0:
                      load_more[0].click()
                      page += 1
                      review_page_soup = scrape_review_page(c_review_link)
                    else:
                      break
                except Exception as e:
                    print(f"An error occurred while loading more: {str(e)}")
                    break

                for review_item in review_page_soup:
                    try:
                        review_titles = review_item.findAll('a', {'class': 'title'})
                        review_texts = review_item.findAll('div', {'class': 'text'})

                        if len(review_titles) == 0:
                            _records.append([c_order, c_title, c_year, c_link, "No reviews found"])
                        else:
                            for i in range(min(len(review_titles)), len(review_texts)):
                                clean_review_title_text = clean_review_title(review_titles[i].text)
                                _records.append([c_order, c_title, c_year, c_link, clean_review_title_text, review_texts[i].text])
                    except Exception as e:
                        print(f"An error occurred while extracting review data: {str(e)}")
                        continue
    return _records



In [ ]:
import pandas as pd
from urllib.request import urlopen as uRequest

url = 'https://www.imdb.com/search/title/?groups=top_100'
records = scrape_page(url)
data = pd.DataFrame(records, columns=['order', 'title', 'year', 'link', 'review title','review text'])
data.head(10)


An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not callable
An error occurred while loading more: 'NoneType' object is not c

KeyboardInterrupt: ignored

# Save data to CSV

In [ ]:
print('Saving to CSV...')
data.to_csv('Dataset_IMDB.csv', sep=',', encoding='utf-8')
print('Done!')

Saving to CSV...
Done!


# NLP